#### importacion de librerias

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
import re
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import geojson
import folium
from folium.plugins import HeatMap



#### Extracion Transformacion Carga

In [2]:
homicidios = pd.read_excel("Data/homicidios.xlsx")

homicidios

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [3]:
# Verifica y cuenta los valores nulos para cada columna en el DataFrame

homicidios.isnull().sum()


ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [4]:
# Obtiene las columnas con tipos de datos numéricos
columnas_numericas = homicidios.select_dtypes(include=['int', 'float']).columns

# Obtiene las columnas con tipos de datos categóricos
columnas_categoricas = homicidios.select_dtypes(include=['object']).columns

# Imprime las variables numéricas
print("Variables Numéricas:")
print(columnas_numericas)

# Imprime las variables categóricas
print("\nVariables Categóricas:")
#print(columnas_categoricas)

Variables Numéricas:
Index(['N_VICTIMAS', 'AAAA', 'MM', 'DD', 'Altura', 'COMUNA'], dtype='object')

Variables Categóricas:


In [5]:
# inspecion de columnas 

homicidios[["Altura", "Calle", "Dirección Normalizada", "Cruce"]][:5]

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN
3,NaN,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL."
4,NaN,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES."


In [6]:
# Filtra las filas donde el valor en la columna "Dirección Normalizada" es nulo

homicidios[homicidios["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce", "Altura", "Calle"]]


,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


In [7]:
columnas_analizadas = ["Altura", "Calle", "Dirección Normalizada", "Cruce"]

# Calcula el porcentaje de valores nulos para cada columna y lo imprime
for columna in columnas_analizadas:
    porcentaje_nulos = (homicidios[columna].isnull().sum() / len(homicidios)) * 100
    print(f"Porcentaje de valores nulos en '{columna}': {porcentaje_nulos:.2f}%")


Porcentaje de valores nulos en 'Altura': 81.47%
Porcentaje de valores nulos en 'Calle': 0.14%
Porcentaje de valores nulos en 'Dirección Normalizada': 1.15%
Porcentaje de valores nulos en 'Cruce': 24.57%


In [8]:
# Se elimina la columna "Altura" ya que el porcentaje de nulos es muy elevado y por el dato que representa no es conveniente hacer una imputacion.

homicidios.drop("Altura", axis=1)


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,RIESTRA AV.,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,GAONA AV.,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,"PERON, EVA AV.",NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [9]:
# Verifica y cuenta la cantidad de filas duplicadas

homicidios.duplicated().sum()

0

In [10]:
homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [11]:
# Accede a la columna "HH" y obtiene los valores únicos
homicidios["HH"].unique()


array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

In [12]:

# Convierte la columna "HH" del DataFrame 'homicidios' a tipo de dato numérico
# con el parámetro 'errors="coerce"', convierte los valores no numéricos a NaN.

homicidios["HH"] = pd.to_numeric(homicidios["HH"], errors="coerce")

# Convierte la columna "HH" a tipo de dato entero nullable (Int64).
# Esto permite manejar valores enteros con la posibilidad de tener NaN.

homicidios["HH"] = homicidios["HH"].astype("Int64")


In [13]:
# Crea una serie booleana 'correcciones' que indica si las columnas 'pos x' y 'pos y'
# contienen caracteres diferentes a números, comas, puntos o guiones.
correcciones = ~(homicidios['pos x'].str.contains('[^0-9.,-]') | homicidios['pos y'].str.contains('[^0-9.,-]') | (homicidios['pos x'] == '.') | (homicidios['pos y'] == '.'))

# En el DataFrame 'homicidios', establece los valores en 'pos x' y 'pos y' a NaN
# en las filas donde la serie 'correcciones' es False (es decir, donde no hay errores).
homicidios.loc[~correcciones, ['pos x', 'pos y']] = np.nan

# Reemplaza las comas por puntos en las columnas 'pos x' y 'pos y',
# luego convierte estas columnas a tipo de dato flotante.
homicidios['pos x'] = homicidios['pos x'].str.replace(',', '.').astype(float)
homicidios['pos y'] = homicidios['pos y'].str.replace(',', '.').astype(float)


In [14]:
homicidios

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,AUTO,CARGAS


In [15]:
# Definición de la función para extraer coordenadas de un texto
def coordenadas(texto):
    # Utiliza una expresión regular para encontrar todas las coincidencias de números decimales en el texto
    coordenadas = re.findall(r'\d+\.\d+', texto)
    
    # Verifica si hay exactamente 2 valores de coordenadas y los convierte a flotantes
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        # Si no hay 2 valores de coordenadas, retorna None para ambas coordenadas
        return None, None

# Aplicar la función a la columna 'XY (CABA)' del DataFrame 'homicidios'
homicidios['coordenada_x'], homicidios['coordenada_y'] = zip(*homicidios['XY (CABA)'].map(coordenadas))

# Filtrar los registros que tienen ambas coordenadas no nulas
homicidios = homicidios[(homicidios['coordenada_x'].notnull()) & (homicidios['coordenada_y'].notnull())]


In [16]:
homicidios

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,99840.652248,94269.165344
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO,102728.600901,98186.249292
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO,99624.297958,97569.698011
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS,99116.454924,101045.232848
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,AUTO,CARGAS,99324.544640,97676.269324


In [17]:
# Capitaliza la primera letra con el metodo "capitalize"
homicidios.columns = [column.capitalize() for column in homicidios.columns]

# Reemplaza los guiones bajos (_) con espacios en blanco en los nombres de las columnas
homicidios.columns = homicidios.columns.str.replace("_", " ")

# Renombra específicamente algunas columnas en el DataFrame 'homicidios'
homicidios = homicidios.rename(columns={
    "N victimas": "Num víctimas",       
    "Aaaa": "Año",                     
    "Mm": "Mes",                        
    "Dd": "Día",                       
    "Hh": "Hora_sin_minutos"            
})


In [18]:
homicidios.head(2)

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora,Hora_sin_minutos,Lugar del hecho,Tipo de calle,...,Dirección normalizada,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420


In [19]:
# Añade una nueva columna 'Día_de_la_semana' 
# que contiene el nombre del día de la semana correspondiente a la columna 'Fecha'

homicidios['Día_de_la_semana'] = homicidios['Fecha'].dt.strftime('%A')

# Un diccionario que mapea los nombres en inglés de los días de la semana
# a sus equivalentes en español.
dic = {
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo',
    'Thursday': 'Jueves',
    'Monday': 'Lunes',
    'Wednesday': 'Miércoles',
    'Tuesday': 'Martes'
}

# Reemplaza los nombres de los días de la semana en la columna 'Día_de_la_semana'
# con sus equivalentes en español utilizando el diccionario 'dic'.
homicidios['Día_de_la_semana'] = homicidios['Día_de_la_semana'].replace(dic)


In [20]:
homicidios

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora,Hora_sin_minutos,Lugar del hecho,Tipo de calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Día_de_la_semana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sábado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,99840.652248,94269.165344,Domingo
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158,Jueves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO,102728.600901,98186.249292,Lunes
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO,99624.297958,97569.698011,Lunes
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS,99116.454924,101045.232848,Jueves
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,9,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,AUTO,CARGAS,99324.544640,97676.269324,Miércoles


#### Extracion Transformacion Carga: homicidios_victimas

In [21]:
homicidios_victimas = pd.read_excel("Data/homicidios.xlsx", "VICTIMAS")

homicidios_victimas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00


In [22]:
homicidios_victimas.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

In [23]:
# Verifica y cuenta los valores nulos para cada columna en el DataFrame

homicidios_victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [24]:
homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [25]:
# Convierte la columna "EDAD" en el DataFrame 'homicidios_victimas' a tipo de dato numérico
# utilizando el método pd.to_numeric. Los valores no numéricos se convierten a NaN.

homicidios_victimas["EDAD"] = pd.to_numeric(homicidios_victimas["EDAD"], errors="coerce")

# Convierte la columna "EDAD" en el DataFrame 'homicidios_victimas' a tipo de dato entero nullable (Int64)
# utilizando el método astype. Esto permite manejar valores enteros con la posibilidad de tener NaN.

homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].astype("Int64")


In [26]:
# Reemplaza los valores en la columna "EDAD" 
# con los valores [" ", "SD"] por NaN. Esto se hace para tratar los valores vacíos o no disponibles.


homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].replace([" ","SD"], np.nan)

In [27]:
# Rellena los valores NaN en la columna "EDAD" 
# con el valor 0.0 utilizando el método fillna.

homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].fillna(0.0)

In [28]:
# Obtiene los valores únicos de la columna "EDAD" 
# utilizando el método unique().

homicidios_victimas ["EDAD"].unique()

<IntegerArray>
[19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20, 54,  0,
 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40, 87, 76, 75, 35, 80,
 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39, 28,  7, 48,  4, 82, 32, 17,
 47, 61, 10, 95, 73, 84, 66, 85,  1, 15, 13, 77, 44, 31, 62, 74, 71, 11, 86,
 69, 72, 12, 63, 92, 68, 91, 64,  5, 88]
Length: 86, dtype: Int64

In [29]:
# Convierte la columna "FECHA_FALLECIMIENTO" 
# a tipo de dato datetime utilizando el método pd.to_datetime.
# El parámetro 'format' especifica el formato de la fecha en los datos originales.

homicidios_victimas["FECHA_FALLECIMIENTO"] = pd.to_datetime(homicidios_victimas["FECHA_FALLECIMIENTO"], errors='coerce', format='%Y-%m-%d')

In [30]:

# Extrae la parte de la fecha (ignorando la información de la hora) y convierte la columna "FECHA_FALLECIMIENTO"
# al tipo de dato 'date'.
homicidios_victimas["FECHA_FALLECIMIENTO"] = homicidios_victimas["FECHA_FALLECIMIENTO"].dt.date

# Reemplaza los valores NaT (Not a Time) con una cadena vacía en la columna "FECHA_FALLECIMIENTO"

homicidios_victimas["FECHA_FALLECIMIENTO"] = homicidios_victimas["FECHA_FALLECIMIENTO"].replace(pd.NaT, "")


In [31]:
homicidios_victimas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02


In [32]:
# Calcula la suma de valores nulos para cada columna 

homicidios_victimas.isnull().sum()


ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [33]:
# Define los límites de los intervalos (bins) para los rangos etarios.
bins = [0, 20, 40, 60, 80, 100]

# Etiquetas que se asignarán a cada intervalo de edad.
labels = ['Menos de 20', 'Entre 21-40', 'Entre 41-60', 'Entre 61-80', 'Más de 80']

# Utiliza la función cut para asignar a cada edad en la columna 'EDAD' un rango etario según los bins y labels definidos.
# El parámetro 'right=False' indica que los intervalos son semiabiertos en el lado derecho (incluyen el valor inferior pero excluyen el superior).
homicidios_victimas['Rango_etario'] = pd.cut(homicidios_victimas['EDAD'], bins=bins, labels=labels, right=False)



In [34]:
homicidios_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Rango_etario
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,Menos de 20
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,Entre 61-80
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,Entre 21-40
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,,Menos de 20
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,Entre 21-40


In [35]:
# Convierte todos los nombres de columnas a formato de título (primera letra en mayúscula, resto en minúscula).
homicidios_victimas.columns = [x.capitalize() for x in homicidios_victimas.columns]

# Reemplaza guiones bajos (_) con espacios en blanco en los nombres de las columnas.
homicidios_victimas.columns = homicidios_victimas.columns.str.replace('_', ' ')

# Renombra específicamente algunas columnas 
homicidios_victimas = homicidios_victimas.rename(columns={'Id hecho': 'Id',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Victima':'Víctima'})


In [36]:
homicidios_victimas.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Víctima', 'Sexo', 'Edad',
       'Fecha fallecimiento', 'Rango etario'],
      dtype='object')

In [37]:
# eliminan las columnas repetidas para realizar la union con el dataset homicidios

homicidios_victimas = homicidios_victimas.drop(['Fecha', 'Año', 'Mes', 'Día', 'Víctima'], axis=1)

homicidios_victimas.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario'], dtype='object')

In [38]:
# Combina los DataFrames 'homicidios_victimas' y 'homicidios' utilizando la columna 'Id' como clave de combinación.
# La combinación se realiza de manera izquierda ('left'), lo que significa que se conservan todas las filas de 'homicidios_victimas'.

accidentes = homicidios_victimas.merge(homicidios, on="Id", how="left")
accidentes.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario',
       'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora_sin_minutos', 'Lugar del hecho', 'Tipo de calle', 'Calle',
       'Altura', 'Cruce', 'Dirección normalizada', 'Comuna', 'Xy (caba)',
       'Pos x', 'Pos y', 'Participantes', 'Victima', 'Acusado', 'Coordenada x',
       'Coordenada y', 'Día_de_la_semana'],
      dtype='object')

In [39]:
accidentes.head(2)

,Id,Rol,Sexo,Edad,Fecha fallecimiento,Rango etario,Num víctimas,Fecha,Año,Mes,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Día_de_la_semana
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,Menos de 20,1.0,2016-01-01,2016.0,1.0,...,8.0,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,Entre 61-80,1.0,2016-01-02,2016.0,1.0,...,9.0,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sábado


In [40]:
# muestra los valores unicos de la columna "Comuna"

accidentes["Comuna"].unique()

array([ 8.,  9.,  1., 11., 15.,  4.,  7., 12.,  3., 13., nan, 14., 10.,
        6.,  2.,  5.])

In [41]:
# Aplica una función lambda a cada fila del DataFrame 'accidentes' para modificar el formato de la columna 'Comuna'.
accidentes["Comuna"] = accidentes["Comuna"].apply(lambda x: f"Comuna N°{int(x) if pd.notna(x) else ''}")


In [42]:
accidentes

,Id,Rol,Sexo,Edad,Fecha fallecimiento,Rango etario,Num víctimas,Fecha,Año,Mes,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Día_de_la_semana
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,Menos de 20,1.0,2016-01-01,2016.0,1.0,...,Comuna N°8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,Entre 61-80,1.0,2016-01-02,2016.0,1.0,...,Comuna N°9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sábado
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03,Entre 21-40,1.0,2016-01-03,2016.0,1.0,...,Comuna N°1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo
3,2016-0004,CONDUCTOR,MASCULINO,18,,Menos de 20,1.0,2016-01-10,2016.0,1.0,...,Comuna N°8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,99840.652248,94269.165344,Domingo
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01,Entre 21-40,1.0,2016-01-21,2016.0,1.0,...,Comuna N°1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158,Jueves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,PEATON,FEMENINO,50,2021-12-12,Entre 41-60,1.0,2021-12-12,2021.0,12.0,...,Comuna N°3,Point (105258.35368554 102122.93231400),-58.405969,-34.610120,PEATON-AUTO,PEATON,AUTO,105258.353686,102122.932314,Domingo
713,2021-0093,PASAJERO_ACOMPAÑANTE,FEMENINO,18,2021-12-18,Menos de 20,1.0,2021-12-13,2021.0,12.0,...,Comuna N°7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO,102728.600901,98186.249292,Lunes
714,2021-0094,PASAJERO_ACOMPAÑANTE,FEMENINO,43,2021-12-20,Entre 41-60,1.0,2021-12-20,2021.0,12.0,...,Comuna N°9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO,99624.297958,97569.698011,Lunes
715,2021-0095,CONDUCTOR,MASCULINO,27,2022-01-02,Entre 21-40,1.0,2021-12-30,2021.0,12.0,...,Comuna N°11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS,99116.454924,101045.232848,Jueves


In [43]:
# Muestra un resumen de la información del DataFrame 'accidentes', incluyendo la cantidad de valores no nulos, tipos de datos y uso de memoria.

accidentes.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 717 entries, 0 to 716
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   Rol                    717 non-null    object        
 2   Sexo                   717 non-null    object        
 3   Edad                   717 non-null    Int64         
 4   Fecha fallecimiento    717 non-null    object        
 5   Rango etario           717 non-null    category      
 6   Num víctimas           702 non-null    float64       
 7   Fecha                  702 non-null    datetime64[ns]
 8   Año                    702 non-null    float64       
 9   Mes                    702 non-null    float64       
 10  Día                    702 non-null    float64       
 11  Hora                   702 non-null    object        
 12  Hora_sin_minutos       701 non-null    Int64         
 13  Lugar

In [44]:
# Convierte la columna 'Día' al tipo de datos entero (Int64).

accidentes["Día"] = accidentes["Día"].astype("Int64")


In [45]:
# muestra los valores unicos de la columna "Dia"

accidentes["Día"].unique()

<IntegerArray>
[   1,    2,    3,   10,   21,   24,   29,    8,   14,   15,   17,   28,    4,
   12,   13,   19,   23,   30,   31,   11, <NA>,   20,   22,   25,   26,   16,
   18,   27,    7,    9,    5,    6]
Length: 32, dtype: Int64

In [46]:
# Convierte la columna 'Mes' del DataFrame 'accidentes' al tipo de datos entero (Int64).

accidentes["Mes"]= accidentes["Mes"].astype ("Int64")

# Obtiene los valores únicos en la columna 'Mes' después de la conversión de tipo.

accidentes["Mes"].unique()

<IntegerArray>
[1, 2, 3, 4, <NA>, 5, 6, 7, 8, 9, 10, 11, 12]
Length: 13, dtype: Int64

In [47]:
# Convierte la columna 'Año' del DataFrame 'accidentes' al tipo de datos entero (Int64).

accidentes["Año"]= accidentes["Año"].astype ("Int64")

# Obtiene los valores únicos en la columna 'Año' después de la conversión de tipo.

accidentes["Año"].unique()

<IntegerArray>
[2016, <NA>, 2017, 2018, 2019, 2020, 2021]
Length: 7, dtype: Int64

In [48]:
# muestra los valores unicos de la columna "Num_victimas"

accidentes["Num víctimas"].unique()

array([ 1.,  2., nan,  3.])

In [49]:
# Rellena los valores nulos en la columna 'Num víctimas' del DataFrame con ceros.
accidentes["Num víctimas"] = accidentes["Num víctimas"].fillna(0.0)

# Convierte la columna 'Num víctimas' a tipo de datos entero (Int64) después de la manipulación de valores nulos.
accidentes["Num víctimas"] = accidentes["Num víctimas"].astype("Int64")


In [50]:
# muestra los valores unicos de la columna "Num_victimas" despues de la conversion

accidentes["Num víctimas"].unique()

<IntegerArray>
[1, 2, 0, 3]
Length: 4, dtype: Int64

In [51]:
accidentes.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario',
       'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora_sin_minutos', 'Lugar del hecho', 'Tipo de calle', 'Calle',
       'Altura', 'Cruce', 'Dirección normalizada', 'Comuna', 'Xy (caba)',
       'Pos x', 'Pos y', 'Participantes', 'Victima', 'Acusado', 'Coordenada x',
       'Coordenada y', 'Día_de_la_semana'],
      dtype='object')

In [52]:
# muestra el tipo de dato de la columna "Fecha"

accidentes["Fecha"].dtype

dtype('<M8[ns]')

In [53]:
# Extrae la parte de la fecha (sin la información de la hora) y actualiza la columna 'Fecha'.

accidentes["Fecha"] = accidentes["Fecha"].dt.date


In [54]:
# Genera estadísticas descriptivas para las columnas numéricas del DataFrame 'accidentes'.

accidentes.describe()

,Edad,Num víctimas,Año,Mes,Día,Hora_sin_minutos,Altura,Pos x,Pos y,Coordenada x,Coordenada y
count,717.000000,717.000000,702.000000,702.000000,702.000000,701.000000,133.000000,702.000000,702.000000,702.000000,702.000000
mean,39.050209,1.037657,2018.192308,6.693732,15.927350,11.573466,3384.187970,-58.441710,-34.619609,101980.113004,101070.596233
std,22.010669,0.294226,1.669154,3.584551,8.657478,6.667562,3095.161836,0.046383,0.035291,4253.717694,3914.938863
min,0.000000,0.000000,2016.000000,1.000000,1.000000,0.000000,30.000000,-58.529942,-34.705250,93889.867899,91571.233073
25%,24.000000,1.000000,2017.000000,3.250000,9.000000,6.000000,1366.000000,-58.476995,-34.643801,98744.378735,98387.965943
50%,35.000000,1.000000,2018.000000,7.000000,16.000000,11.000000,2639.000000,-58.444513,-34.622835,101721.590022,100711.696729
75%,55.000000,1.000000,2019.000000,10.000000,23.000000,17.000000,4500.000000,-58.401892,-34.596969,105630.884244,103582.742374
max,95.000000,3.000000,2021.000000,12.000000,31.000000,23.000000,16080.000000,-58.356082,-34.534654,109831.098614,110496.053085


In [55]:
accidentes.rename(columns={'Día_de_la_semana': 'Dias de la semana'}, inplace=True)


In [56]:
# guarda el dataset en la carpeta Data

accidentes.to_csv("Data/homicidios_limpio.csv", index=False)